### Para comenzar con el scrapping, importamos las funciones que tenemos en el archivo 'procesos'

In [1]:
from procesos import *

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import re

### Establecemos el nombre de las columnas que tendrá el dataframe

In [3]:
columnas = ['Obra','Status','Ejecución física %','Ejecución financiera %','Estado','Municipio','Dirección','Personas afectadas','Ente responsable','Funcionario responsable','Empresa contratista','Valor total de la obra USD (millones)','Obra financiada por FONDEN','Monto comprometido USD (millones)','Fecha estimada de inicio','Fecha real de inicio','Fecha culminación estimada de la obra','Fecha de paralización']

### Llamamos a la función 'obtener_datos_obra' que realiza el proceso de web scraping (selenium), obtenemos los datos que queremos de la obra y lo convertimos en un dataframe

In [4]:
url = 'https://transparenciave.org/obrasinconclusas/obras_inconclusas/conexion-colector-aguas-servidas-a-red-principal/'
conex_aguas_resid = obtener_datos_obra(url)

In [6]:
df = pd.DataFrame([conex_aguas_resid]) #convertimos los datos en un dataframe

df['Obra financiada por FONDEN'] = 'S/I' #agregamos las columnas que hacen falta para mantener el formato del DF
df['Monto comprometido USD (millones)'] = 'S/I'

#Reorganizamos el orden de las columnas
df = df[[0,1,2,3,4,5,6,7,8,9,10,11,'Obra financiada por FONDEN','Monto comprometido USD (millones)',12,13,14,15]]
df.columns = columnas
df

,Obra,Status,Ejecución física %,Ejecución financiera %,Estado,Municipio,Dirección,Personas afectadas,Ente responsable,Funcionario responsable,Empresa contratista,Valor total de la obra USD (millones),Obra financiada por FONDEN,Monto comprometido USD (millones),Fecha estimada de inicio,Fecha real de inicio,Fecha culminación estimada de la obra,Fecha de paralización
0,CONEXIÓN COLECTOR AGUAS SERVIDAS A RED PRINCIPAL,Operativa sin concluir las obras,70%,S/I,Estado Lara,Iribarren,"Barrio Rómulo Gallegos, parroquía El Cují",Entre 5.001 y 100.000,Hidrolara / Gobernación del estado Lara,Henri Falcón,S/I,USD 1.222.222,S/I,S/I,2015,2015,2018,2017


In [14]:
url2 = 'https://transparenciave.org/obrasinconclusas/obras_inconclusas/construccion-planta-de-tratamiento-de-aguas-residuales-el-tanque/'
planta_tto = obtener_datos_obra(url2)

In [16]:
df2 = pd.DataFrame([planta_tto])

df2 = df2.drop(df2.columns[[2, 3]], axis=1) #eliminamos las columnas 2 y 3 que no tienen información

df2['Obra financiada por FONDEN'] = 'S/I' #agregamos las columnas que hacen falta para mantener el formato del DF
df2['Monto comprometido USD (millones)'] = 'S/I'

#Reorganizamos el orden de las columnas
df2 = df2[[0,1,4,5,6,7,8,9,10,11,12,13,'Obra financiada por FONDEN','Monto comprometido USD (millones)',14,16,16,17]]
df2.columns = columnas

In [18]:
url3 = 'https://transparenciave.org/obrasinconclusas/obras_inconclusas/proyecto-de-saneamiento-del-rio-guaire-fases-ii-y-iii/'
rio_guiare = obtener_datos_obra(url3)

In [21]:
df3 = pd.DataFrame([rio_guiare])
df3.columns = columnas

In [22]:
url4 = 'https://transparenciave.org/obrasinconclusas/obras_inconclusas/relleno-sanitario-canaveral/'
canaveral = obtener_datos_obra(url4)

In [26]:
df4 = pd.DataFrame([canaveral])

df4['Obra financiada por FONDEN'] = 'S/I' #agregamos las columnas que hacen falta para mantener el formato del DF
df4['Monto comprometido USD (millones)'] = 'S/I'

#Reorganizamos el orden de las columnas
df4 = df4[[0,1,2,3,4,5,6,7,8,9,10,11,'Obra financiada por FONDEN','Monto comprometido USD (millones)',12,13,14,15]]
df4.columns = columnas

In [27]:
url5 = 'https://transparenciave.org/obrasinconclusas/obras_inconclusas/saneamiento-integral-del-valle-del-turbio/'
valle_turbio = obtener_datos_obra(url5)

In [29]:
df5 = pd.DataFrame([rio_guiare])
df5.columns = columnas

### Procedemos a juntar todos los dataframes

In [44]:
# Concatena los DataFrames en uno solo
data_saneamiento = pd.concat([df,df2,df3,df4,df5],ignore_index=True)

### Sustituimos en la columna 'Estados' donde aparezca 'Área metropolitana de Caracas', por 'Distrito Capital' que es el verdadero nombre del estado

In [45]:
data_saneamiento['Estado'] = data_saneamiento['Estado'].replace('Área Metropolitana de Caracas', 'Distrito Capital')

### Para conocer las coordenadas de cada una de las obras, hacemos uso de Geopy

In [46]:
#Asignamos como dirección el Municipio y Estado de cada obra para conocer sus coordenadas
municipio_estado = data_saneamiento['Municipio'] + ', ' + data_saneamiento['Estado']
lista_municipio_estado = municipio_estado.tolist()

In [47]:
latitudes = obtener_latitudes(lista_municipio_estado) #obtenemos la latitud de la obra

In [48]:
longitudes = obtener_longitudes(lista_municipio_estado) #obtenemos la longitud de la obra

In [49]:
data_saneamiento['Latitud'] = latitudes #agregamos una nueva columna con todos los valores de las latitudes
data_saneamiento['Longitud'] = longitudes #agregamos una nueva columna con todos los valores de las longitudes

### Extraemos la parte numérica de la columna 'Valor total de la obra' para tener un dato de tipo entero

In [50]:
# Aplicar la función a la columna 'Valor total de la obra'
data_saneamiento['Valor total de la obra USD (millones)'] = data_saneamiento['Valor total de la obra USD (millones)'].apply(extract_numeric_value)

data_saneamiento['Valor total de la obra USD (millones)'] = data_saneamiento['Valor total de la obra USD (millones)'].str.replace('.', '', regex=True).fillna('0').astype(int)

#Dividimos para tener el total de millones en un número mas legible
data_saneamiento['Valor total de la obra USD (millones)'] = (data_saneamiento['Valor total de la obra USD (millones)'] / 1000000).round(2)

### Limpiamos la columna 'Monto comprometido', extraemos la parte numérica, llenamos los valores nulos con '0', transformamos a un tipo de dato float

In [52]:
data_saneamiento['Monto comprometido USD (millones)'] = data_saneamiento['Monto comprometido USD (millones)'].apply(extract_numeric_value)
data_saneamiento['Monto comprometido USD (millones)'] = data_saneamiento['Monto comprometido USD (millones)'].fillna('0')

# Convertir la columna a cadena, eliminar los puntos y reemplazar las comas por puntos
data_saneamiento['Monto comprometido USD (millones)'] = data_saneamiento['Monto comprometido USD (millones)'].astype(str)
data_saneamiento['Monto comprometido USD (millones)'] = data_saneamiento['Monto comprometido USD (millones)'].str.replace('.', '').str.replace(',', '.')

# Convertir la columna a tipo float
data_saneamiento['Monto comprometido USD (millones)'] = pd.to_numeric(data_saneamiento['Monto comprometido USD (millones)'], errors='coerce')

#Dividimos para tener el total de millones en un número más legible
data_saneamiento['Monto comprometido USD (millones)'] = (data_saneamiento['Monto comprometido USD (millones)'] / 1000000).astype(int)

### Cambiamos el tipo de dato de la columna 'Fecha estimada de inicio' por datetime y extraemos el año

In [53]:
# Convertir la columna 'Fecha estimada de inicio' a datetime
data_saneamiento['Fecha estimada de inicio'] = pd.to_datetime(data_saneamiento['Fecha estimada de inicio'], errors='coerce')
data_saneamiento['Fecha estimada de inicio'] = data_saneamiento['Fecha estimada de inicio'].dt.year

### Transformamos las columnas 'Ejecución física' y 'Ejecución financiera' en valores de tipo float, para ello llenamos los valores 'S/I' con 0 y extraemos la parte numérica

In [55]:
data_saneamiento['Ejecución física %'] = data_saneamiento['Ejecución física %'].apply(extract_numeric_value)
data_saneamiento['Ejecución física %'] = data_saneamiento['Ejecución física %'].str.replace(',', '.').replace({None: '0'})

# Convertir la columna a tipo float
data_saneamiento['Ejecución física %'] = data_saneamiento['Ejecución física %'].astype(float)

In [56]:
data_saneamiento['Ejecución financiera %'] = data_saneamiento['Ejecución financiera %'].apply(extract_numeric_value)
data_saneamiento['Ejecución financiera %'] = data_saneamiento['Ejecución financiera %'].str.replace(',', '.').replace({None: '0'})

# Convertir la columna a tipo float
data_saneamiento['Ejecución financiera %'] = data_saneamiento['Ejecución financiera %'].astype(float)

### Añadimos una nueva columna para indicar cuál es el sector correspondiente a la obra

In [57]:
data_saneamiento['Sector'] = 'Saneamiento ambiental'

In [58]:
#Reordenamos las columnas
nuevo_orden = ['Sector','Obra','Status','Ejecución física %','Ejecución financiera %','Estado','Municipio','Dirección','Personas afectadas','Ente responsable','Funcionario responsable','Empresa contratista','Valor total de la obra USD (millones)','Obra financiada por FONDEN','Monto comprometido USD (millones)','Fecha estimada de inicio','Fecha real de inicio','Fecha culminación estimada de la obra','Fecha de paralización', 'Latitud', 'Longitud']
data_saneamiento = data_saneamiento[nuevo_orden]

In [59]:
data_saneamiento

,Sector,Obra,Status,Ejecución física %,Ejecución financiera %,Estado,Municipio,Dirección,Personas afectadas,Ente responsable,...,Empresa contratista,Valor total de la obra USD (millones),Obra financiada por FONDEN,Monto comprometido USD (millones),Fecha estimada de inicio,Fecha real de inicio,Fecha culminación estimada de la obra,Fecha de paralización,Latitud,Longitud
0,Saneamiento ambiental,CONEXIÓN COLECTOR AGUAS SERVIDAS A RED PRINCIPAL,Operativa sin concluir las obras,70.0,0.0,Estado Lara,Iribarren,"Barrio Rómulo Gallegos, parroquía El Cují",Entre 5.001 y 100.000,Hidrolara / Gobernación del estado Lara,...,S/I,1.22,S/I,0,2015.0,2015,2018,2017,10.123217,-69.408749
1,Saneamiento ambiental,CONSTRUCCIÓN PLANTA DE TRATAMIENTO DE AGUAS RE...,Paralizada,33.0,0.0,Estado Lara,Palavecino,"Las cercanías del caserío El Taque, Palavecino...",Más de 1.000.001,Empresa Noroccidental de Mantenimiento y Obras...,...,S/I,63.48,S/I,0,NaN,S/I,S/I,2009,9.987491,-69.233509
2,Saneamiento ambiental,"PROYECTO DE SANEAMIENTO DEL RÍO GUAIRE, FASES ...",Paralizada,41.7,0.0,Distrito Capital,Libertador,Río Guaire,Más de 1.000.001,Ministerio de Atención de las Aguas,...,S/I,2500.00,Si,103,2005.0,2007,2016,S/I,10.501021,-66.960280
3,Saneamiento ambiental,RELLENO SANITARIO CAÑAVERAL,Paralizada,45.0,0.0,Estado Bolívar,Caroní,"7439+X93, 8050",Entre 0-5.000,Ministerio de Ecosocialismo / Secretaría de Am...,...,"Pavimentos Delta, C.A. (Pavidelca)",405.08,S/I,0,2014.0,Julio 2016,Noviembre 2016,S/I,8.360815,-62.634737
4,Saneamiento ambiental,"PROYECTO DE SANEAMIENTO DEL RÍO GUAIRE, FASES ...",Paralizada,41.7,0.0,Distrito Capital,Libertador,Río Guaire,Más de 1.000.001,Ministerio de Atención de las Aguas,...,S/I,2500.00,Si,103,2005.0,2007,2016,S/I,10.501021,-66.960280


### Exportamos nuestro dataframe

In [60]:
data_saneamiento.to_csv('obras_saneamiento.csv', index=False)